In [96]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

from VR_algorithm import VR_algorithm
from cost_func import *
from read_data import *

import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
X, Y = read_mnist(datatype = 'multiclass', mask_label = [0, 1, 2, 3, 4, 5, 6, 7 ,8 ,9])

Extracting /Users/kunyuan/Documents/Github/dist_avrg/Distributed_VR_algorithm/C:/Users/biche/OneDrive/Documents/Python Scripts/random_reshuffle/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/kunyuan/Documents/Github/dist_avrg/Distributed_VR_algorithm/C:/Users/biche/OneDrive/Documents/Python Scripts/random_reshuffle/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/kunyuan/Documents/Github/dist_avrg/Distributed_VR_algorithm/C:/Users/biche/OneDrive/Documents/Python Scripts/random_reshuffle/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/kunyuan/Documents/Github/dist_avrg/Distributed_VR_algorithm/C:/Users/biche/OneDrive/Documents/Python Scripts/random_reshuffle/MNIST_data/t10k-labels-idx1-ubyte.gz
Extracting /Users/kunyuan/Documents/Github/dist_avrg/Distributed_VR_algorithm/C:/Users/biche/OneDrive/Documents/Python Scripts/random_reshuffle/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/kunyuan/Documents/Github/dist_avrg/Distributed_VR_algorithm/C:/Users/biche

In [98]:
N = X.shape[0]
D = X.shape[1]
C = Y.shape[1]

N_train_prctg, N_val_prctg = 0.4, 0.1
N_train = int(N * N_train_prctg)
N_val = int(N * N_val_prctg)

X_train, Y_train = X[0:N_train, :], Y[0:N_train, :]
X_val, Y_val = X[N_train: N_train+N_val, :], Y[N_train: N_train+N_val, :]
X_test, Y_test = X[N_train+N_val:N+1, :], Y[N_train+N_val:N+1, :]

y_train, y_val, y_test = np.argmax(Y_train, axis = 1), np.argmax(Y_val, axis = 1), np.argmax(Y_test, axis = 1)

In [99]:
print Y_test.shape

(27500, 10)


In [100]:
print Y_train[0:10,:]

[[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]]


# Train the model

In [104]:
N_epoch = 10
rho = 1e-4
w_star = np.random.rand(C*D, 1) * 0.1
params= {'using_sgd': 1, 'epoch_per_FG': 2, 'minibatch': 1}

In [105]:
avrg = VR_algorithm(X_train, Y_train , w_star, soft_max, rho = rho)
MSD, ER = avrg.train(N_epoch=N_epoch, mu=.5, method='SVRG', replace = False, **params)
w_svrg = avrg.cost_model.w

epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10


In [106]:
W_svrg = w_svrg.reshape(D, C, order = 'F')
y_train_pred = np.argmax(X_train.dot(W_svrg), axis = 1)
print ('training accuracy: %f') % (np.mean(y_train == y_train_pred) )
y_val_pred = np.argmax(X_val.dot(W_svrg), axis = 1)
print ('validation accuracy: %f') % (np.mean(y_val == y_val_pred) )

training accuracy: 0.907045
validation accuracy: 0.883455


In [107]:
y_test_pred = np.argmax(X_test.dot(W_svrg), axis = 1)
print ('test accuracy: %f') % (np.mean(y_test == y_test_pred) )

test accuracy: 0.902727
